# PROJECT TITLE:
Store sales -Time series forecasting 

In [1]:
#Data handling 
import pandas as pd 
import numpy as np
 
from dotenv import dotenv_values 
import pyodbc
 

In [ ]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')
# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVERNAME")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")
connection_string = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"